In [2]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [1]:
xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'

In [3]:
from symred.chem import PySCFDriver

In [4]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [5]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [6]:
pyscf_obj.run_pyscf()

In [7]:
2*pyscf_obj.pyscf_hf.mol.nao

8

In [8]:
from symred.chem import FermionicHamilt

In [9]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 2.29310124732
((0, 1), (0, 0)) -1.8385126260957712
((0, 1), (4, 0)) 0.1535200531800066
((1, 1), (1, 0)) -1.8385126260957712
((1, 1), (5, 0)) 0.1535200531800066
((2, 1), (2, 0)) -1.5485198738611377
((2, 1), (6, 0)) 0.1346528866850458
((3, 1), (3, 0)) -1.5485198738611377
((3, 1), (7, 0)) 0.1346528866850458
((4, 1), (0, 0)) 0.15352005318000683
((4, 1), (4, 0)) -1.2423978239696827
((5, 1), (1, 0)) 0.15352005318000683
((5, 1), (5, 0)) -1.2423978239696827
((6, 1), (2, 0)) 0.13465288668504574
((6, 1), (6, 0)) -0.9084576464931442
((7, 1), (3, 0)) 0.13465288668504574
((7, 1), (7, 0)) -0.9084576464931442
((0, 1), (0, 1), (0, 0), (0, 0)) 0.2504327702741458
((0, 1), (0, 1), (0, 0), (4, 0)) -0.04165187456702536
((0, 1), (0, 1), (2, 0), (2, 0)) 0.07725837026056045
((0, 1), (0, 1), (2, 0), (6, 0)) -0.02286714657234941
((0, 1), (0, 1), (4, 0), (0, 0)) -0.041651874567025335
((0, 1), (0, 1), (4, 0), (4, 0)) 0.053084040051590224
((0, 1), (0, 1), (6, 0), (2, 0)) -0.022867146572349395
((0, 1), (0, 1), (

In [10]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 0, 0, 0, 0])

In [30]:
hf_ket = H_ferm.hf_ket
H_mat = H_ferm.get_sparse_ham()

In [12]:
from pyscf.cc.addons import spatial2spin

# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [14]:
# singles



t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T
ccsd_single_amps.shape

(8, 8)

In [20]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[4, 0, 5, 1], -0.03338374667042344],
 [[4, 0, 6, 2], -0.015763962924379395],
 [[4, 0, 7, 3], -0.040635820882763435],
 [[4, 1, 5, 0], 0.03338374667042344],
 [[4, 1, 7, 2], 0.02487185795838404],
 [[4, 2, 5, 3], -0.09739744775359767],
 [[4, 2, 6, 0], 0.015763962924379395],
 [[4, 2, 7, 1], -0.02487185795838404],
 [[4, 3, 5, 2], 0.09739744775359767],
 [[4, 3, 7, 0], 0.040635820882763435],
 [[5, 0, 4, 1], 0.03338374667042344],
 [[5, 0, 6, 3], 0.02487185795838404],
 [[5, 1, 4, 0], -0.03338374667042344],
 [[5, 1, 6, 2], -0.040635820882763435],
 [[5, 1, 7, 3], -0.015763962924379395],
 [[5, 2, 4, 3], 0.09739744775359767],
 [[5, 2, 6, 1], 0.040635820882763435],
 [[5, 3, 4, 2], -0.09739744775359767],
 [[5, 3, 6, 0], -0.02487185795838404],
 [[5, 3, 7, 1], 0.015763962924379395],
 [[6, 0, 4, 2], 0.015763962924379395],
 [[6, 0, 5, 3], -0.02487185795838404],
 [[6, 0, 7, 1], -0.02720605738917505],
 [[6, 1, 5, 2], 0.040635820882763435],
 [[6, 1, 7, 0], 0.02720605738917505],
 [[6, 2, 4, 0], -0.015763962

In [19]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[4, 0], 0.004359835192369033],
 [[4, 2], -8.975874218963638e-17],
 [[5, 1], 0.004359835192369033],
 [[5, 3], -8.975874218963638e-17],
 [[6, 0], 1.712784237893248e-16],
 [[6, 2], -0.002633112813562081],
 [[7, 1], 1.712784237893248e-16],
 [[7, 3], -0.002633112813562081]]

In [21]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [23]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [37]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))

In [41]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

(-2.1663896084345176+0j)

In [42]:
pyscf_obj.pyscf_ccsd.e_tot

-2.1663896084345176